In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv('retail_train.csv')


In [3]:
data.head(10)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
6,2375,26984851516,1,1043142,1,1.57,364,-0.68,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
8,2375,26984851516,1,1102651,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,6423775,1,2.00,364,-0.79,1642,1,0.0,0.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2396804 entries, 0 to 2396803
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   user_id            int64  
 1   basket_id          int64  
 2   day                int64  
 3   item_id            int64  
 4   quantity           int64  
 5   sales_value        float64
 6   store_id           int64  
 7   retail_disc        float64
 8   trans_time         int64  
 9   week_no            int64  
 10  coupon_disc        float64
 11  coupon_match_disc  float64
dtypes: float64(4), int64(8)
memory usage: 219.4 MB


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [6]:
result_test = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns=['user_id', 'actual']

In [7]:
result_test.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [8]:
# making train DataFrame with users and their real purchases
result_train = data_train.groupby('user_id')['item_id'].unique().reset_index()
result_train.columns=['user_id', 'actual']

In [9]:
result_train.head(5)

,user_id,actual
0,1,"[825123, 831447, 840361, 845307, 852014, 85498..."
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55..."
2,3,"[866211, 878996, 882830, 904360, 921345, 93194..."
3,4,"[836163, 857849, 877523, 878909, 883932, 89142..."
4,5,"[938983, 5980822, 1012352, 825538, 1002499, 69..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
### - Можно сэмплировать товары случайно, но пропорционально какому-либо весу

two kinds of weights: calculated on 'sales_value' and on 'quantity'


In [10]:
# df is dataframe with a column 'item_id' and relevant event column ('quantity' or 'sales_value')
def get_probability(df):
    items = df.item_id
    event_series = df[(df.columns[1])]
    weight_series = np.around(event_series/event_series.sum(), decimals=9)
    
# here should be Exception handling
    if np.around(weight_series.sum())==1:
        # to make the sum of all weights = 1
        remainder = weight_series.sum() - 1     
        weight_series[0] = weight_series[0] - remainder
       
        df['item_weight'] = weight_series
# return df with a new column 'item_weight' containing event-derived probability for each item
        return df
    else:
        return np.nan

In [11]:
def weighted_random_recommendation(df, n=5):
   # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    items = df.item_id
    item_p = df.item_weight
    
    recs = np.random.choice(items, size=n, replace=False, p=item_p)  
   
    return recs.tolist()

In [12]:
# getting new df with a column 'item_id' and relevant event column 'sales_value'
item_sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()
item_sales.head(2)

,item_id,sales_value
0,25671,20.94
1,26081,0.99


In [13]:
len(item_sales)

86865

In [14]:
# new df with a column 'item_id' and relevant event column 'quantity'
item_quantity = data_train.groupby('item_id')['quantity'].sum().reset_index()
item_quantity.head(2)

,item_id,quantity
0,25671,6
1,26081,1


In [15]:
#getting weights from 'sales_value'
item_sales = get_probability(item_sales)
item_sales.head(2)

,item_id,sales_value,item_weight
0,25671,20.94,1.274000e-06
1,26081,0.99,1.400000e-07


In [16]:
#getting weights from 'quantity'
item_quantity = get_probability(item_quantity)
item_quantity.head(2)

,item_id,quantity,item_weight
0,25671,6,8.348000e-06
1,26081,1,4.000000e-09


Getting recommendations on test data by random sampling with weights from 'sales_value' column 

In [17]:
%%time

# your_code
result_test['rand_rec_sales'] = result_test['user_id'].apply(lambda x: weighted_random_recommendation(item_sales, n=5))
result_test.head(5)

Wall time: 2.99 s


,user_id,actual,rand_rec_sales
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569230, 5564067, 8019216, 1118120, 5568378]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6533889, 6534178, 876232, 15596908, 12324954]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[901287, 851895, 6533765, 5585068, 1135768]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[7025193, 1082185, 903230, 6391551, 921504]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[12672980, 5564569, 6534178, 901062, 1133018]"


Getting recommendations on test data by random sampling with weights from 'quantity' column 

In [18]:
%%time

# your_code
result_test['rand_rec_quant'] = result_test['user_id'].apply(lambda x: weighted_random_recommendation(item_quantity, n=5))
result_test.head(5)

Wall time: 5.02 s


,user_id,actual,rand_rec_sales,rand_rec_quant
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569230, 5564067, 8019216, 1118120, 5568378]","[6534178, 6534166, 6410464, 6533889, 6544236]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6533889, 6534178, 876232, 15596908, 12324954]","[6534178, 6534166, 6533889, 827667, 962568]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[901287, 851895, 6533765, 5585068, 1135768]","[5668996, 6534178, 6533889, 397896, 6544236]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[7025193, 1082185, 903230, 6391551, 921504]","[6534178, 6534166, 5703832, 6533889, 1404121]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[12672980, 5564569, 6534178, 901062, 1133018]","[6534178, 6533889, 1404121, 826249, 6534166]"


In [19]:
#being not ML algorithm, the weighted random recommendation can be tried on train data, too

result_train['rand_rec_sales'] = result_train['user_id'].apply(lambda x: weighted_random_recommendation(item_sales, n=5))

In [20]:
result_train['rand_rec_quant'] = result_train['user_id'].apply(lambda x: weighted_random_recommendation(item_quantity, n=5))

In [21]:
result_train.head(5)

,user_id,actual,rand_rec_sales,rand_rec_quant
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[967205, 13512426, 12385477, 1028926, 12812261]","[6534178, 6534166, 1426702, 6544236, 6533889]"
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[828420, 6534178, 956902, 1127831, 873654]","[6534166, 6534178, 6533889, 1404121, 1101174]"
2,3,"[866211, 878996, 882830, 904360, 921345, 93194...","[909894, 12577242, 1029743, 849843, 992475]","[6533889, 6534166, 6534178, 397896, 6544236]"
3,4,"[836163, 857849, 877523, 878909, 883932, 89142...","[12524691, 1113381, 1020993, 1106301, 911401]","[6534178, 6544236, 5703832, 6533889, 6534166]"
4,5,"[938983, 5980822, 1012352, 825538, 1002499, 69...","[1064892, 824072, 882829, 1000237, 994262]","[6534178, 7441102, 6534166, 6533889, 397896]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [22]:
# your_code
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #assuming top-k items are in the beginning of the list
    flags = np.isin(bought_list, recommended_list[:k])
    
    precision = flags.sum() / k
    
    
    return precision

In [23]:
# evaluating recommendations on train data

In [24]:
result_test.apply(lambda row: precision_at_k(row['rand_rec_sales'], row['actual']), axis=1).mean()

0.019588638589618023

In [25]:
result_test.apply(lambda row: precision_at_k(row['rand_rec_quant'], row['actual']), axis=1).mean()

0.047992164544564155

In [26]:
# evaluating recommendations on train data

In [27]:
result_train.apply(lambda row: precision_at_k(row['rand_rec_sales'], row['actual']), axis=1).mean()

0.09875950380152061

In [28]:
result_train.apply(lambda row: precision_at_k(row['rand_rec_quant'], row['actual']), axis=1).mean()

0.13925570228091236

### Conclusion
Quantity column seems to be a better weight provider, than sales_value column.
They both seem to be better, than random recommendation without weights (lesson1 precision@5 = 0.0009794319294809011),   
but worse than popular recommendation and ML algorithms

Interesting - worked better on train, though I didn't expect it.

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [29]:
# your_code
